In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from langchain_milvus.utils.sparse import BM25SparseEmbedding
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from tqdm.notebook import tqdm

# from threading import Thread
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda')


In [3]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            # device_map='auto',
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            # temperature=0
            ).to(device)
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')
             
    def _call(self, infor,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # response, history = self.model.chat(self.tokenizer, prompt, history=[])
        # print('infor:', infor)
        # system_info =  infor.split('说“谢谢你的提问！”。')[-1].split('问题：')[0].strip()
        # human_info =  infor.split('问题：')[-1].strip()
        # time.sleep(1000)
        messages = [
                    # {'role':'system', 'content': system_info},
                    {'role':'user', 'content': infor}
                    ]
        # print('打印messages内容：', messages)
        # print('打印messages类型：', type(messages))
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors='pt').to(device)
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=1024
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
       
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [4]:
def init_db():
    # persist_directory = r'vectordb/chroma/m3e-base'
    persist_directory = r'vectordb/milvus/bag-large/milvus_demo.db'
    # embeddings_model_cache_path = r'autodl-tmp/embedding_model/AI-ModelScope/m3e-base'
    
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/BAAI/bge-large-zh-v1___5'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    
    # 加载缓存知识库
    vectordb = Milvus(
        connection_args={'uri': persist_directory},
        embedding_function=embeddings,
    )
    return vectordb

In [5]:
def init_model(vectordb):
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    template = """
        使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
        {context}
        问题：{question}
        """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=['context', 'question'], template=template)

    # 构造检索问答链
    qa_chain_ = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(
            search_type="mmr", 
            search_kwargs={"k": 10}
        # search_kwargs={"k": 10, "fetch_k": 50, 'index_type': "mmr", 'metric_type':'L2'}
        ),
        # retriever = vectordb.max_marginal_relevance_search_by_vector(limit=10)
        return_source_documents=True,
        chain_type_kwargs={'prompt': QA_CHAIN_PROMPT},
        # chain_type='stuff'
    )
    return qa_chain_

In [6]:
def init2_model(vectordb):
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir=model_cache_path)
    # template = """
    #     使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
    #     上下文：{context}
    #     问题：{question}
    #     回答：
    #     """
    template = """
    分析输入的上下文，段落之间利用空行进行分隔，对每一个段落与问题进行相似度匹配，找出与问题相似度高的段落文本，并直接将文本输出。
    上下文：{context}
    问题：{question}
    请不要作答，直接输出匹配的文本。
    """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=['context', 'question'], template=template)

    qa_chain_ = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT)

    return qa_chain_, llm

In [7]:
# 加载数据
import pandas as pd
import csv
import os
# import re
# from tqdm import tqdm
# 初始化模型
vectordb = init_db()
model = init_model(vectordb)

正从本地加载模型。。。。。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

模型加载完成！


In [8]:
datas_ = pd.read_excel('QA_test.xlsx') 
# datas_ = datas[datas['问题分类'] == '政策内容']Question	Answer	Text	Title

questions = datas_['Question']
titles = datas_['Title']
# labels = datas_['问题分类']
# bodys = datas_['提取的实体']
file = 'milvus召回测试-L2.csv'
for question, title in tqdm(zip(questions, titles), ncols=len(questions)):
# question = '意向受让方在登记受让意向时需要满足哪些条件？'

    response = model.invoke({'query': question})
    source_title = [doc.metadata['title'] for doc in response['source_documents']]
    # source_title = [file_.strip() for file_ in source_title]
    # print('问题：', question)
    # print('回答：', response['result'])
    # print('来源文档：', source_title)
    for tt in range(len(source_title)):
        if title == source_title[tt]:
            calling_ = 'True'
            calling_loc = tt
            break
        else:
            calling_ = 'False'
            calling_loc = '无'
        
   
    data_ = [question, source_title, title, calling_, calling_loc]
    # print(data_)
    with open(file, 'a', newline='', encoding='utf-8') as f:
        fw = csv.writer(f)
        if not os.path.getsize(file):
            header = ['问题', '召回文档标题', '真实标题', '正确召回', '召回位置']
            fw.writerow(header)
        fw.writerow(data_)

# print('文档内容', response['source_documents'])
# time.sleep(1000)
# print('问题：', question)
# print('回答：', response['result'])
# print('来源文档：', response['source_documents'])

0it [00:00, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
datas = pd.read_csv('实体提取_改进.csv') 
datas_ = datas[datas['问题分类'] == '政策内容']
questions = datas_['问题']
labels = datas_['问题分类']
bodys = datas_['提取的实体']
# question = '意向受让方,登记受让意向,条件'
# 测试用实体来召回文本，并用召回的文本和问题回答
for question, body_ in tqdm(zip(questions, bodys), ncols=len(questions)):
    # 这里可能要将召回的问答链进行拆分，分两步
    question = '开标时，投标文件是如何被处理的？'
    # body_ = '上海市规划和自然资源局关于印发《国有建设用地使用权招标拍卖挂牌出让投标竞买外汇保证金账户管理制度》的通知'
    # body_ = '投标人,竞买人,违约,保证金,处理'
    body_ = '开标，投标文件'
    call_data = vectordb.as_retriever(
        search_type="mmr", 
        search_kwargs={"k": 10, "fetch_k": 20}).get_relevant_documents(body_ )
    # print(call_data)
    print([doc.metadata['title'] for doc in call_data])
    page_contents = ''
    for doc in call_data:
        page_contents += doc.page_content.strip() + '\n\n'

    # 提取文本摘要呢？
    response = model.run({'context': page_contents, 'question': question})
    print('匹配response:', response)
    print('++++'*100)
    time.sleep(1000)
    # print('问题：', question)
    # print('回答：', response['result'])
    # print('来源文档：', response['source_documents'])

In [ ]:
'''
第一阶段召回改善方式
0、扩大召回数目；
1、提取问题关键词、实体， 每个实体随意组合以及单个实体进行多次召回；
2、对问题进行重新生成多种说法，召回多次；（备选，因为我们在做意图识别时会进行实体提取，如果提取成功率较高的话，这一步就可以省略，没必要生成多个问题）；
3、rerank排序召回，考虑第1、2点的多次召回重复较多的文档title作为一个权重排序；
'''

In [ ]:
# persist_directory = r'vectordb/chroma/m3e-base'
persist_directory = r'vectordb/milvus/bag-large/milvus_demo.db'
# embeddings_model_cache_path = r'autodl-tmp/embedding_model/AI-ModelScope/m3e-base'

embeddings_model_cache_path = r'autodl-tmp/embedding_model/BAAI/bge-large-zh-v1___5'
# 加载词向量模型
embeddings = HuggingFaceEmbeddings(
    model_name=embeddings_model_cache_path)

# 加载缓存知识库
vectordb = Milvus(
    connection_args={'uri': persist_directory},
    embedding_function=embeddings,
)

In [9]:
source_title = [doc.metadata['title'] for doc in response['source_documents']]

In [10]:
source_title

['招标公告和公示信息发布管理办法',
 '铁路工程建设项目招标投标管理办法',
 '政府采购货物和服务招标投标管理办法',
 '水运工程建设项目招标投标管理办法',
 '民航局机场司关于印发《民航专业工程建设项目招标投标管理办法》的通知',
 '招标公告和公示信息发布管理办法',
 '工程建设项目施工招标投标办法',
 '招标公告和公示信息发布管理办法',
 '财政部关于做好政府采购信息公开工作的通知',
 '工程建设项目货物招标投标办法']

In [9]:
if title in source_title:
    print(True)

True
